<a href="https://colab.research.google.com/github/fjadidi2001/AD_Prediction/blob/main/AD%D9%80Ad1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip '/content/drive/MyDrive/db_al/Q1 Dataset.zip'

unzip:  cannot find or open /content/drive/MyDrive/db_al/Q1 Dataset.zip, /content/drive/MyDrive/db_al/Q1 Dataset.zip.zip or /content/drive/MyDrive/db_al/Q1 Dataset.zip.ZIP.


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [4]:
np.random.seed(42)
tf.random.set_seed(42)

In [5]:
data_dir = '/content/Alzheimer_s_Disease_Neuroimaging_ADNI_Dataset'

# Load data
def load_data(data_dir):
    images, labels = [], []
    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        for image_file in os.listdir(label_dir):
            image_path = os.path.join(label_dir, image_file)
            image = tf.keras.preprocessing.image.load_img(image_path, target_size=(128, 128))
            image = tf.image.resize(image, (64,64))
            image = tf.keras.preprocessing.image.img_to_array(image)
            images.append(image)
            labels.append(label)
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

images, labels = load_data(data_dir)
labels = np.where(labels == 'AD', 1, 0)
one_hot = np.zeros((labels.shape[0], 2))
one_hot[np.arange(labels.shape[0]), labels] = 1
labels = one_hot

NameError: name 'os' is not defined

In [ ]:
# Train-validation-test split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.05, random_state=42, stratify=labels)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, stratify=y_train)

In [ ]:
# Display class distribution before augmentation
sns.countplot(x=np.argmax(y_train, 1))
plt.title('Class Distribution Before Augmentation')
plt.show()

In [ ]:
batch_size = 64

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size)
test_generator = test_datagen.flow(X_test, y_test, batch_size=1)

# Display class distribution after augmentation
augmented_images, _ = next(train_generator)

augmented_labels = []
for _ in range((len(y_train)*5 // batch_size)+1):
    _, batch_labels = next(train_generator)
    augmented_labels.extend(batch_labels)
sns.countplot(x=np.argmax(augmented_labels,1))
plt.title('Class Distribution After Augmentation')
plt.show()

# Display 5 random augmented images
fig, axes = plt.subplots(1, 5, figsize=(20, 5))
for i in range(5):
    random_index = np.random.randint(0, len(augmented_images))
    ax = axes[i]
    ax.imshow(augmented_images[random_index])
    ax.axis('off')
plt.show()

In [ ]:
# Model Definition
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    Dropout(0.2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Dropout(0.2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.GlorotUniform()),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(2, activation='softmax')
])

testing_model_1 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')
])

testing_model_2 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')
])

In [ ]:
checkpoint_path = 'best_model.h5'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_path,
    save_best_only = True,
    monitor = 'val_loss',
    mode = 'min',
    verbose = 1
)

In [ ]:
# Model Compilation
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Model Training
history = model.fit(train_generator,
                    epochs=500,
                    validation_data=val_generator,
                    callbacks=[checkpoint_callback])

# Model Evaluation
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}", f"Test loss: {test_loss}")

# Save Model
model.save('alzheimer_cnn_model.h5')

In [ ]:
def plot_history(history):
    # Plot loss and accuracy history
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))
    # Plot loss
    axs[0].plot(history.history['loss'], label='Train Loss')
    axs[0].plot(history.history['val_loss'], label='Validation Loss')
    axs[0].set_title('Loss')
    axs[0].set_xlabel('Epochs')
    axs[0].set_ylabel('Loss')
    axs[0].legend()
    # Plot accuracy
    axs[1].plot(history.history['accuracy'], label='Train Accuracy')
    axs[1].plot(history.history['val_accuracy'], label='Validation Accuracy')
    axs[1].set_title('Accuracy')
    axs[1].set_xlabel('Epochs')
    axs[1].set_ylabel('Accuracy')
    axs[1].legend()
    plt.show()
plot_history(history)


In [ ]:
# Get true labels
Y_true = []
for _ in range(len(y_test)):
    _, batch_labels = next(test_generator)
    Y_true.extend(batch_labels)
Y_true = np.argmax(Y_true, 1)

In [ ]:
from sklearn.metrics import roc_curve, auc
test_generator.reset()
Y_pred = model.predict(test_generator)
Y_pred_classes = np.argmax(Y_pred, 1)

# Plot ROCcurve
fpr, tpr, _ = roc_curve(Y_true, Y_pred[:,1])
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Confusion matrix
from sklearn.metrics import confusion_matrix
import itertools
conf_matrix = confusion_matrix(Y_true, Y_pred_classes)
plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(2)
plt.xticks(tick_marks, ['Class 0', 'Class 1'], rotation=45)
plt.yticks(tick_marks, ['Class 0', 'Class 1'])
thresh = conf_matrix.max() / 2.
for i, j in itertools.product(range(conf_matrix.shape[0]), range(conf_matrix.shape[1])):
    plt.text(j, i, format(conf_matrix[i, j], 'd'),
         horizontalalignment="center",
         color="white" if conf_matrix[i, j] > thresh else "black")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(Y_true, Y_pred_classes)
precision = precision_score(Y_true, Y_pred_classes)
recall = recall_score(Y_true, Y_pred_classes)
f1 = f1_score(Y_true, Y_pred_classes)
print(f'Accuracy: {accuracy*100:.2f}')
print(f'Precision: {precision*100:.2f}')
print(f'Recall: {recall*100:.2f}')
print(f'F1 Score: {f1:.3f}')
print(f'AUC: {roc_auc:.3f}')
print(f"Loss: {test_loss:.3f}")